In [1]:
1+1

2

In [ ]:
# 라이브러리 import
import os
import mysql.connector
from datetime import datetime
from typing import List, Dict, Any, Optional
import json
import re
import logging

# OpenAI와 LangChain 관련 import
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

# 환경 변수 설정을 위한 import
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 로깅 설정
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "your-openai-api-key-here")

print("✅ 모든 라이브러리 import 완료")

✅ 모든 라이브러리 import 완료


In [3]:
# OpenAI API 키 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key-here")  # 실제 키로 변경하세요
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# MySQL 데이터베이스 연결 설정
DB_CONFIG = {
    "host": os.environ["DB_HOST"],
    "user": os.environ["DB_USER"],
    "password": os.environ["DB_PASSWORD"],
    "database": os.environ["DB_NAME"],
    "charset": "utf8mb4",
}

print("✅ 환경 변수 및 DB 설정 완료")

✅ 환경 변수 및 DB 설정 완료


In [4]:
# 질문 의도 분석을 위한 프롬프트 템플릿 정의
INTENT_ANALYSIS_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""
당신은 청년 정책 관련 질문을 분석하여 MySQL 쿼리를 생성하는 전문가입니다.

사용자 질문: {question}

다음 정보를 JSON 형태로 추출해주세요:
1. 지역 (region): 서울, 경기, 부산 등
2. 연령 (age): 숫자로 표현된 나이
3. 정책 유형 (policy_type): 인턴십, 취업, 창업, 교육, 주거, 생활비 등
4. 학력 (education): 고졸, 대졸, 대학원 등
5. 취업 상태 (employment): 재직, 구직, 창업 등
6. 소득 수준 (income): 저소득, 중간소득 등
7. 키워드 (keywords): 핵심 키워드들의 리스트

응답 형식:
{{
    "region": "지역명 또는 null",
    "age": "나이 또는 null",
    "policy_type": "정책유형 또는 null",
    "education": "학력 또는 null",
    "employment": "취업상태 또는 null",
    "income": "소득수준 또는 null",
    "keywords": ["키워드1", "키워드2", ...]
}}
""",
)

print("✅ 의도 분석 프롬프트 템플릿 생성 완료")

✅ 의도 분석 프롬프트 템플릿 생성 완료


In [5]:
class IntentOutputParser(BaseOutputParser):
    """의도 분석 결과를 파싱하는 커스텀 파서"""

    def parse(self, text: str) -> Dict[str, Any]:
        """JSON 형태의 텍스트를 파싱하여 딕셔너리로 변환"""
        try:
            # JSON 부분만 추출
            json_match = re.search(r"\{.*\}", text, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                return json.loads(json_str)
            else:
                # JSON을 찾지 못한 경우 기본값 반환
                return {
                    "region": None,
                    "age": None,
                    "policy_type": None,
                    "education": None,
                    "employment": None,
                    "income": None,
                    "keywords": [],
                }
        except json.JSONDecodeError:
            print("❌ JSON 파싱 실패")
            return {
                "region": None,
                "age": None,
                "policy_type": None,
                "education": None,
                "employment": None,
                "income": None,
                "keywords": [],
            }


# 파서 인스턴스 생성
intent_parser = IntentOutputParser()
print("✅ 커스텀 파서 생성 완료")

✅ 커스텀 파서 생성 완료


In [7]:
class YouthPolicyQueryGenerator:
    """청년 정책 쿼리 생성기"""

    def __init__(self, openai_api_key: str):
        """초기화"""
        self.llm = OpenAI(
            model="gpt-4o-mini",
            openai_api_key=openai_api_key,
            temperature=0.1,  # 일관성 있는 결과를 위해 낮은 temperature 설정
            max_tokens=1000,
        )
        self.intent_chain = LLMChain(
            llm=self.llm, prompt=INTENT_ANALYSIS_PROMPT, output_parser=intent_parser
        )

    def analyze_intent(self, question: str) -> Dict[str, Any]:
        """질문의 의도를 분석"""
        print(f"🔍 질문 분석 중: {question}")

        try:
            result = self.intent_chain.run(question=question)
            print(f"📊 분석 결과: {result}")
            return result
        except Exception as e:
            print(f"❌ 의도 분석 실패: {str(e)}")
            return {
                "region": None,
                "age": None,
                "policy_type": None,
                "education": None,
                "employment": None,
                "income": None,
                "keywords": [],
            }

    def generate_query(self, intent: Dict[str, Any]) -> str:
        """의도 분석 결과를 바탕으로 SQL 쿼리 생성"""
        print("🛠️ SQL 쿼리 생성 중...")

        # 기본 SELECT 문
        base_query = "SELECT * FROM youth_policies WHERE 1=1"
        conditions = []

        # 지역 조건 (기관명이나 정책명에서 지역 검색)
        if intent.get("region"):
            region = intent["region"]
            conditions.append(
                f"(sprvsn_inst_cd_nm LIKE '%{region}%' OR oper_inst_cd_nm LIKE '%{region}%')"
            )

        # 연령 조건
        if intent.get("age"):
            try:
                age = int(intent["age"])
                conditions.append(
                    f"(sprt_trgt_min_age <= {age} AND sprt_trgt_max_age >= {age})"
                )
            except ValueError:
                pass

        # 정책 유형 조건 (정책명, 설명, 지원내용에서 검색)
        if intent.get("policy_type"):
            policy_type = intent["policy_type"]
            conditions.append(
                f"(plcy_nm LIKE '%{policy_type}%' OR plcy_expln_cn LIKE '%{policy_type}%' OR plcy_sprt_cn LIKE '%{policy_type}%')"
            )

        # 키워드 조건 (모든 키워드에 대해 OR 조건으로 검색)
        if intent.get("keywords") and len(intent["keywords"]) > 0:
            keyword_conditions = []
            for keyword in intent["keywords"]:
                if keyword.strip():  # 빈 키워드 제외
                    keyword_conditions.append(
                        f"(plcy_nm LIKE '%{keyword}%' OR plcy_kywd_nm LIKE '%{keyword}%' OR plcy_expln_cn LIKE '%{keyword}%')"
                    )

            if keyword_conditions:
                conditions.append(f"({' OR '.join(keyword_conditions)})")

        # 조건들을 쿼리에 추가
        if conditions:
            base_query += " AND " + " AND ".join(conditions)

        # 정렬 조건 추가 (최신순)
        base_query += " ORDER BY frst_reg_dt DESC"

        # 결과 제한 (너무 많은 결과 방지)
        base_query += " LIMIT 20"

        print(f"📝 생성된 쿼리: {base_query}")
        return base_query


# 쿼리 생성기 인스턴스 생성
query_generator = YouthPolicyQueryGenerator(OPENAI_API_KEY)
print("✅ 쿼리 생성 클래스 구현 완료")

✅ 쿼리 생성 클래스 구현 완료


In [8]:
def execute_query(query: str, db_config: Dict[str, str]) -> List[Dict[str, Any]]:
    """MySQL 쿼리 실행 함수"""
    print("🔗 데이터베이스 연결 중...")

    try:
        # 데이터베이스 연결
        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor(dictionary=True)

        print(f"📊 쿼리 실행 중: {query}")

        # 쿼리 실행
        cursor.execute(query)
        results = cursor.fetchall()

        print(f"✅ {len(results)}개의 결과를 찾았습니다.")

        return results

    except mysql.connector.Error as e:
        print(f"❌ 데이터베이스 오류: {str(e)}")
        return []

    except Exception as e:
        print(f"❌ 예상치 못한 오류: {str(e)}")
        return []

    finally:
        # 연결 정리
        if "cursor" in locals():
            cursor.close()
        if "connection" in locals():
            connection.close()
        print("🔒 데이터베이스 연결 종료")


print("✅ 데이터베이스 연결 함수 구현 완료")

✅ 데이터베이스 연결 함수 구현 완료


In [9]:
def youth_policy_rag_process(
    question: str, db_config: Dict[str, str]
) -> Dict[str, Any]:
    """온통청년 RAG 프로세스 전체 실행 함수"""
    print("🚀 온통청년 RAG 프로세스 시작")
    print(f"💬 사용자 질문: {question}")
    print("-" * 50)

    # 1. 질문 의도 분석
    intent = query_generator.analyze_intent(question)
    print("-" * 50)

    # 2. SQL 쿼리 생성
    sql_query = query_generator.generate_query(intent)
    print("-" * 50)

    # 3. 쿼리 실행
    results = execute_query(sql_query, db_config)
    print("-" * 50)

    # 4. 결과 반환
    return {
        "question": question,
        "intent": intent,
        "sql_query": sql_query,
        "results": results,
        "result_count": len(results),
    }


print("✅ 통합 RAG 프로세스 함수 구현 완료")

✅ 통합 RAG 프로세스 함수 구현 완료


In [10]:
def display_results(rag_result: Dict[str, Any]) -> None:
    """RAG 결과를 보기 좋게 표시하는 함수"""
    print("📋 RAG 프로세스 결과")
    print("=" * 60)

    print(f"🔍 원본 질문: {rag_result['question']}")
    print(f"📊 분석된 의도: {rag_result['intent']}")
    print(f"🛠️ 생성된 쿼리: {rag_result['sql_query']}")
    print(f"📈 결과 개수: {rag_result['result_count']}")

    print("\n📄 검색 결과:")
    print("-" * 60)

    if rag_result["results"]:
        for i, policy in enumerate(rag_result["results"][:5], 1):  # 상위 5개만 표시
            print(f"\n{i}. {policy.get('plcy_nm', 'N/A')}")
            print(f"   🏢 주관기관: {policy.get('sprvsn_inst_cd_nm', 'N/A')}")
            print(
                f"   🎯 대상연령: {policy.get('sprt_trgt_min_age', 'N/A')}~{policy.get('sprt_trgt_max_age', 'N/A')}세"
            )
            print(f"   📝 설명: {policy.get('plcy_expln_cn', 'N/A')[:100]}...")
            print(f"   🔗 신청URL: {policy.get('aply_url_addr', 'N/A')}")
    else:
        print("❌ 검색 결과가 없습니다.")


print("✅ 결과 표시 함수 구현 완료")

✅ 결과 표시 함수 구현 완료


In [11]:
# 테스트 질문들
test_questions = [
    "서울에 사는 청년인데 인턴십 있어",
    "25살이고 취업 지원 정책 찾고 있어",
    "부산에서 창업하고 싶은데 지원 정책 있나요?",
    "대학생 생활비 지원 정책 알려주세요",
]

print("🧪 테스트 실행 시작")
print("=" * 60)

# 각 질문에 대해 RAG 프로세스 실행
for i, question in enumerate(test_questions, 1):
    print(f"\n🔄 테스트 {i}")
    print("=" * 60)

    try:
        # RAG 프로세스 실행
        result = youth_policy_rag_process(question, DB_CONFIG)

        # 결과 표시
        display_results(result)

    except Exception as e:
        print(f"❌ 테스트 {i} 실패: {str(e)}")

    print("\n" + "=" * 60)

print("✅ 테스트 완료")

🧪 테스트 실행 시작

🔄 테스트 1
🚀 온통청년 RAG 프로세스 시작
💬 사용자 질문: 서울에 사는 청년인데 인턴십 있어
--------------------------------------------------
🔍 질문 분석 중: 서울에 사는 청년인데 인턴십 있어


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33972\18025432.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.intent_chain.run(question=question)


📊 분석 결과: {'region': '서울', 'age': None, 'policy_type': '인턴십', 'education': None, 'employment': None, 'income': None, 'keywords': ['청년', '인턴십']}
--------------------------------------------------
🛠️ SQL 쿼리 생성 중...
📝 생성된 쿼리: SELECT * FROM youth_policies WHERE 1=1 AND (sprvsn_inst_cd_nm LIKE '%서울%' OR oper_inst_cd_nm LIKE '%서울%') AND (plcy_nm LIKE '%인턴십%' OR plcy_expln_cn LIKE '%인턴십%' OR plcy_sprt_cn LIKE '%인턴십%') AND ((plcy_nm LIKE '%청년%' OR plcy_kywd_nm LIKE '%청년%' OR plcy_expln_cn LIKE '%청년%') OR (plcy_nm LIKE '%인턴십%' OR plcy_kywd_nm LIKE '%인턴십%' OR plcy_expln_cn LIKE '%인턴십%')) ORDER BY frst_reg_dt DESC LIMIT 20
--------------------------------------------------
🔗 데이터베이스 연결 중...
📊 쿼리 실행 중: SELECT * FROM youth_policies WHERE 1=1 AND (sprvsn_inst_cd_nm LIKE '%서울%' OR oper_inst_cd_nm LIKE '%서울%') AND (plcy_nm LIKE '%인턴십%' OR plcy_expln_cn LIKE '%인턴십%' OR plcy_sprt_cn LIKE '%인턴십%') AND ((plcy_nm LIKE '%청년%' OR plcy_kywd_nm LIKE '%청년%' OR plcy_expln_cn LIKE '%청년%') OR (plcy_nm LIKE '%인턴십%' O

In [12]:
def test_single_question(question: str) -> None:
    """개별 질문을 테스트하는 함수"""
    print(f"🔍 단일 질문 테스트: {question}")
    print("=" * 60)

    try:
        # RAG 프로세스 실행
        result = youth_policy_rag_process(question, DB_CONFIG)

        # 결과 표시
        display_results(result)

        # 쿼리만 별도로 출력
        print(f"\n📝 생성된 SQL 쿼리:")
        print(result["sql_query"])

    except Exception as e:
        print(f"❌ 테스트 실패: {str(e)}")


# 예제 사용법
print("🎯 예제 질문 테스트")
test_single_question("서울에 사는 청년인데 인턴십 있어")

🎯 예제 질문 테스트
🔍 단일 질문 테스트: 서울에 사는 청년인데 인턴십 있어
🚀 온통청년 RAG 프로세스 시작
💬 사용자 질문: 서울에 사는 청년인데 인턴십 있어
--------------------------------------------------
🔍 질문 분석 중: 서울에 사는 청년인데 인턴십 있어
📊 분석 결과: {'region': '서울', 'age': None, 'policy_type': '인턴십', 'education': None, 'employment': None, 'income': None, 'keywords': ['청년', '인턴십']}
--------------------------------------------------
🛠️ SQL 쿼리 생성 중...
📝 생성된 쿼리: SELECT * FROM youth_policies WHERE 1=1 AND (sprvsn_inst_cd_nm LIKE '%서울%' OR oper_inst_cd_nm LIKE '%서울%') AND (plcy_nm LIKE '%인턴십%' OR plcy_expln_cn LIKE '%인턴십%' OR plcy_sprt_cn LIKE '%인턴십%') AND ((plcy_nm LIKE '%청년%' OR plcy_kywd_nm LIKE '%청년%' OR plcy_expln_cn LIKE '%청년%') OR (plcy_nm LIKE '%인턴십%' OR plcy_kywd_nm LIKE '%인턴십%' OR plcy_expln_cn LIKE '%인턴십%')) ORDER BY frst_reg_dt DESC LIMIT 20
--------------------------------------------------
🔗 데이터베이스 연결 중...
📊 쿼리 실행 중: SELECT * FROM youth_policies WHERE 1=1 AND (sprvsn_inst_cd_nm LIKE '%서울%' OR oper_inst_cd_nm LIKE '%서울%') AND (plcy_nm LIKE '%인턴십

In [13]:
class AdvancedQueryGenerator(YouthPolicyQueryGenerator):
    """고급 쿼리 생성기 - 더 정교한 쿼리 생성"""

    def generate_optimized_query(self, intent: Dict[str, Any]) -> str:
        """최적화된 쿼리 생성"""
        print("🔧 최적화된 쿼리 생성 중...")

        # 기본 SELECT 절 (필요한 컬럼만 선택)
        select_columns = [
            "plcy_no",
            "plcy_nm",
            "plcy_expln_cn",
            "plcy_sprt_cn",
            "sprvsn_inst_cd_nm",
            "oper_inst_cd_nm",
            "aply_url_addr",
            "sprt_trgt_min_age",
            "sprt_trgt_max_age",
            "biz_prd_bgnng_ymd",
            "biz_prd_end_ymd",
            "aply_ymd",
            "frst_reg_dt",
        ]

        base_query = f"SELECT {', '.join(select_columns)} FROM youth_policies WHERE 1=1"
        conditions = []

        # 현재 날짜 기준으로 유효한 정책만 필터링
        current_date = datetime.now().strftime("%Y%m%d")
        conditions.append(
            f"(biz_prd_end_ymd >= '{current_date}' OR biz_prd_end_ymd IS NULL OR biz_prd_end_ymd = '')"
        )

        # 승인된 정책만 필터링
        conditions.append("plcy_aprv_stts_cd = 'APPROVED'")

        # 나머지 조건들은 부모 클래스와 동일한 로직 사용
        if intent.get("region"):
            region = intent["region"]
            conditions.append(
                f"(sprvsn_inst_cd_nm LIKE '%{region}%' OR oper_inst_cd_nm LIKE '%{region}%')"
            )

        if intent.get("age"):
            try:
                age = int(intent["age"])
                conditions.append(
                    f"(sprt_trgt_min_age <= {age} AND sprt_trgt_max_age >= {age})"
                )
            except ValueError:
                pass

        if intent.get("policy_type"):
            policy_type = intent["policy_type"]
            conditions.append(
                f"(plcy_nm LIKE '%{policy_type}%' OR plcy_expln_cn LIKE '%{policy_type}%' OR plcy_sprt_cn LIKE '%{policy_type}%')"
            )

        if intent.get("keywords") and len(intent["keywords"]) > 0:
            keyword_conditions = []
            for keyword in intent["keywords"]:
                if keyword.strip():
                    keyword_conditions.append(
                        f"(plcy_nm LIKE '%{keyword}%' OR plcy_kywd_nm LIKE '%{keyword}%' OR plcy_expln_cn LIKE '%{keyword}%')"
                    )

            if keyword_conditions:
                conditions.append(f"({' OR '.join(keyword_conditions)})")

        # 조건들을 쿼리에 추가
        if conditions:
            base_query += " AND " + " AND ".join(conditions)

        # 정렬 조건 (관련성과 최신성 고려)
        base_query += " ORDER BY frst_reg_dt DESC, inq_cnt DESC"

        # 결과 제한
        base_query += " LIMIT 20"

        print(f"🎯 최적화된 쿼리: {base_query}")
        return base_query


# 고급 쿼리 생성기 인스턴스 생성
advanced_generator = AdvancedQueryGenerator(OPENAI_API_KEY)
print("✅ 고급 쿼리 생성기 구현 완료")

✅ 고급 쿼리 생성기 구현 완료


In [14]:
def final_integration_test():
    """최종 통합 테스트"""
    print("🎯 최종 통합 테스트 시작")
    print("=" * 80)

    # 테스트 케이스
    test_case = {
        "question": "서울에 사는 25살 청년인데 인턴십 프로그램 있어?",
        "expected_keywords": ["서울", "25", "청년", "인턴십"],
    }

    print(f"📝 테스트 질문: {test_case['question']}")
    print(f"🎯 예상 키워드: {test_case['expected_keywords']}")
    print("-" * 80)

    try:
        # 1. 기본 RAG 프로세스 실행
        print("🔄 기본 RAG 프로세스 실행")
        basic_result = youth_policy_rag_process(test_case["question"], DB_CONFIG)

        print(f"📊 기본 결과: {basic_result['result_count']}개 정책 발견")
        print(f"🛠️ 기본 쿼리: {basic_result['sql_query']}")

        # 2. 고급 쿼리 생성기로 의도 분석
        print("\n🔧 고급 쿼리 생성기 실행")
        intent = advanced_generator.analyze_intent(test_case["question"])
        optimized_query = advanced_generator.generate_optimized_query(intent)

        print(f"🎯 최적화된 쿼리: {optimized_query}")

        # 3. 최적화된 쿼리 실행
        optimized_results = execute_query(optimized_query, DB_CONFIG)

        print(f"📈 최적화된 결과: {len(optimized_results)}개 정책 발견")

        # 4. 결과 비교
        print("\n📊 결과 비교")
        print(f"기본 결과: {basic_result['result_count']}개")
        print(f"최적화된 결과: {len(optimized_results)}개")

        return {
            "basic_result": basic_result,
            "optimized_result": {
                "question": test_case["question"],
                "intent": intent,
                "sql_query": optimized_query,
                "results": optimized_results,
                "result_count": len(optimized_results),
            },
        }

    except Exception as e:
        print(f"❌ 최종 테스트 실패: {str(e)}")
        return None


# 최종 테스트 실행
final_result = final_integration_test()

if final_result:
    print("\n✅ 최종 통합 테스트 완료")
    print("🎉 온통청년 RAG 시스템이 성공적으로 구축되었습니다!")
else:
    print("\n❌ 최종 통합 테스트 실패")

🎯 최종 통합 테스트 시작
📝 테스트 질문: 서울에 사는 25살 청년인데 인턴십 프로그램 있어?
🎯 예상 키워드: ['서울', '25', '청년', '인턴십']
--------------------------------------------------------------------------------
🔄 기본 RAG 프로세스 실행
🚀 온통청년 RAG 프로세스 시작
💬 사용자 질문: 서울에 사는 25살 청년인데 인턴십 프로그램 있어?
--------------------------------------------------
🔍 질문 분석 중: 서울에 사는 25살 청년인데 인턴십 프로그램 있어?
📊 분석 결과: {'region': '서울', 'age': 25, 'policy_type': '인턴십', 'education': None, 'employment': None, 'income': None, 'keywords': ['인턴십', '청년', '서울', '25살']}
--------------------------------------------------
🛠️ SQL 쿼리 생성 중...
📝 생성된 쿼리: SELECT * FROM youth_policies WHERE 1=1 AND (sprvsn_inst_cd_nm LIKE '%서울%' OR oper_inst_cd_nm LIKE '%서울%') AND (sprt_trgt_min_age <= 25 AND sprt_trgt_max_age >= 25) AND (plcy_nm LIKE '%인턴십%' OR plcy_expln_cn LIKE '%인턴십%' OR plcy_sprt_cn LIKE '%인턴십%') AND ((plcy_nm LIKE '%인턴십%' OR plcy_kywd_nm LIKE '%인턴십%' OR plcy_expln_cn LIKE '%인턴십%') OR (plcy_nm LIKE '%청년%' OR plcy_kywd_nm LIKE '%청년%' OR plcy_expln_cn LIKE '%청년%') OR (plcy_nm